In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download ucsc-cse-164-spring-2022-final-project --force
!unzip ucsc-cse-164-spring-2022-final-project

mv: cannot stat 'kaggle.json': No such file or directory
 99% 82.0M/83.2M [00:01<00:00, 58.1MB/s]
100% 83.2M/83.2M [00:01<00:00, 48.0MB/s]
Archive:  ucsc-cse-164-spring-2022-final-project.zip
replace MedMNIST/lableed/0/0bq8dx5psrcg1w2he9f7.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
ylabel_dict = {}
for i, line in enumerate(open("./MedMNIST/wnids.txt", "r")):
  ylabel_dict[line.rstrip("\n")] = i


In [ ]:
print(ylabel_dict)

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}


In [ ]:
from glob import glob
from PIL import Image
import numpy as np
unlabeled = glob("MedMNIST/unlabeled/*")

unlabeled_images = []
untrue_labels = []
for i in unlabeled:
  img = np.array(Image.open(i).convert("RGB"))
  unlabeled_images.append(img)
  untrue_labels.append(0)

In [ ]:
import re
labeled = glob("MedMNIST/lableed/*/*")
labeled_images = []
label = []
for i in labeled:
  labeled_images.append(np.array(Image.open(i).convert("RGB")))
  match_str = re.match("(\S+)\/(\S+)\/(\S+)", i)
  l = ylabel_dict[(match_str[2])]
  label.append(l)

print(len(label))
print(len(labeled_images))

60
60


In [ ]:
test = glob("MedMNIST/test/*")
test_images = []
for i in test:
  test_images.append(np.array(Image.open(i).convert("RGB")))

In [ ]:
labeled_images = np.array(labeled_images)
test_images = np.array(test_images)
untrue_labels = np.array(untrue_labels)
unlabeled_images = np.array(unlabeled_images)
label = np.array(label)
unlabeled_images = unlabeled_images.astype("float32") / 255

x_train = labeled_images.astype("float32") / 255
y_train = np.array(label)
x_test = test_images.astype("float32") / 255


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
num_classes = 5
shape = (64, 64, 3)
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 500


In [ ]:
resnet_model = keras.Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(64,64,3),
                   pooling='avg',classes=5,
                   weights='imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(keras.layers.Flatten())
resnet_model.add(keras.layers.Dense(units = 1024, activation = 'relu'))
resnet_model.add(keras.layers.BatchNormalization())
resnet_model.add(keras.layers.Dropout(0.5))
resnet_model.add(keras.layers.Dense(512, activation='relu'))
resnet_model.add(keras.layers.BatchNormalization())
resnet_model.add(keras.layers.Dropout(0.5))
resnet_model.add(keras.layers.Dense(256, activation='relu'))
resnet_model.add(keras.layers.BatchNormalization())
resnet_model.add(keras.layers.Dropout(0.5))
resnet_model.add(keras.layers.Dense(128, activation='relu'))
resnet_model.add(keras.layers.BatchNormalization())
resnet_model.add(keras.layers.Dropout(0.5))
resnet_model.add(keras.layers.Dense(100, activation = 'softmax'))

In [ ]:
resnet_model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
resnet_model.fit(labeled_images, y_train, 
                           epochs=20,
                          )

Epoch 1/20
2/2 [==============================] - 4s 23ms/step - loss: 0.9024 - accuracy: 0.8833
Epoch 2/20
2/2 [==============================] - 0s 21ms/step - loss: 0.8650 - accuracy: 0.8667
Epoch 3/20
2/2 [==============================] - 0s 21ms/step - loss: 0.7698 - accuracy: 0.9000
Epoch 4/20
2/2 [==============================] - 0s 25ms/step - loss: 0.6789 - accuracy: 0.9333
Epoch 5/20
2/2 [==============================] - 0s 22ms/step - loss: 0.7020 - accuracy: 0.9000
Epoch 6/20
2/2 [==============================] - 0s 22ms/step - loss: 0.6355 - accuracy: 0.9333
Epoch 7/20
2/2 [==============================] - 0s 22ms/step - loss: 0.5936 - accuracy: 0.9833
Epoch 8/20
2/2 [==============================] - 0s 23ms/step - loss: 0.5716 - accuracy: 0.9500
Epoch 9/20
2/2 [==============================] - 0s 23ms/step - loss: 0.6353 - accuracy: 0.9333
Epoch 10/20
2/2 [==============================] - 0s 22ms/step - loss: 0.6511 - accuracy: 0.9500
Epoch 11/20
2/2 [============

In [ ]:

unlabeled_images = []
untrue_labels = []
for i in unlabeled:
  img = np.array(Image.open(i).convert("RGB"))
  unlabeled_images.append(img)
  untrue_labels.append(0)
unlabeled_images = np.array(unlabeled_images)


In [ ]:
y_prob = resnet_model.predict(unlabeled_images) 

In [ ]:
y_classes = y_prob.argmax(axis=-1)

In [ ]:
counter = 0
wanted_images = []
wanted_labels = []
for i in y_prob:
  if max(i) >= 0.9:
    wanted_images.append(unlabeled_images[counter])
    wanted_labels.append(y_classes[counter])
  counter +=1

In [ ]:
print(len(wanted_images))
print(len(wanted_labels))
wanted_images = np.array(wanted_images)
wanted_labels = np.array(wanted_labels)

36504
36504


In [ ]:
labeled_images = np.append(labeled_images, wanted_images, axis=0)
y_train = np.append(y_train, wanted_labels)

In [ ]:
print(labeled_images.shape)
print(y_train.shape)

(36564, 64, 64, 3)
(36564,)


In [ ]:
resnet_model = keras.Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(64,64,3),
                   pooling='avg',classes=5,
                   weights='imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(keras.layers.Flatten())
resnet_model.add(keras.layers.Dense(units = 1024, activation = 'relu'))
resnet_model.add(keras.layers.BatchNormalization())
resnet_model.add(keras.layers.Dropout(0.5))
resnet_model.add(keras.layers.Dense(512, activation='relu'))
resnet_model.add(keras.layers.BatchNormalization())
resnet_model.add(keras.layers.Dropout(0.5))
resnet_model.add(keras.layers.Dense(256, activation='relu'))
resnet_model.add(keras.layers.BatchNormalization())
resnet_model.add(keras.layers.Dropout(0.5))
resnet_model.add(keras.layers.Dense(128, activation='relu'))
resnet_model.add(keras.layers.BatchNormalization())
resnet_model.add(keras.layers.Dropout(0.5))
resnet_model.add(keras.layers.Dense(100, activation = 'softmax'))
resnet_model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
resnet_model.fit(labeled_images, y_train, 
                           epochs=5,
                          )

Epoch 1/5
1143/1143 [==============================] - 29s 22ms/step - loss: 0.3092 - accuracy: 0.9466
Epoch 2/5
1143/1143 [==============================] - 24s 21ms/step - loss: 0.0239 - accuracy: 0.9962
Epoch 3/5
1143/1143 [==============================] - 24s 21ms/step - loss: 0.0168 - accuracy: 0.9967
Epoch 4/5
1143/1143 [==============================] - 24s 21ms/step - loss: 0.0115 - accuracy: 0.9979
Epoch 5/5
1143/1143 [==============================] - 24s 21ms/step - loss: 0.0092 - accuracy: 0.9984


In [ ]:
print(len(test_images))
test_prob = resnet_model.predict(test_images) 
test_classes = test_prob.argmax(axis=-1)

6000


In [ ]:
image_names = []
for i in test:
  match = re.match("(\S+)\/(\S+)\/(\S+)\.(\S+)", i)
  image_names.append(match[3])
image_names[0]

'i4rn5o79agkvwt61jmql'

In [ ]:
import pandas as pd
df = pd.read_csv("MedMNIST/samples.csv")

for i in range(len(test_classes)):
  df.loc[i, 'image_id'] = image_names[i]
  df.loc[i, 'label'] = test_classes[i]
  

df.to_csv("MedMNIST/samples.csv", index=False)

In [ ]:
resnet_model.save("./content/MEDMNIST")

INFO:tensorflow:Assets written to: ./content/MEDMNIST/assets


Note: I trained the original model (without unlabeled data) like 4 times. I stopped when accuracy was close to 1. So if ur gonna run the code again, you have to run that block 4 times.